In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline
symbols = ['GOOGL', 'TSLA', 'FB', 'AMZN', 'AAPL', 'MSFT', 'VOD',  'ADBE', 'NVDA', 'CRM' ]
all_stocks = pd.DataFrame()
for symbol in symbols:
    tmp_close = yf.download(symbol, 
                      start='2020-11-07', 
                      end='2020-12-07', 
                      progress=False)['Close']
    all_stocks = pd.concat([all_stocks, tmp_close], axis=1)
all_stocks.columns=symbols
all_stocks

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/site-packages/yfinance/multi.py", line 167, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/site-packages/yfinance/multi.py", line 179, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/Users/sergiogago/opt/anaconda3/envs/qiskit24/lib/python3.8/site-packages/yf

KeyboardInterrupt: 

In [ ]:
returns = np.log(all_stocks/all_stocks.shift(1)).dropna(how="any")
returns.plot(figsize=(12,10))


In [ ]:
# Optimizer to minimize the risk
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(returns.cov()).dot(weights.T)
    
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.
        # The expected daily return of our portfolio and we want to be at greater than 0.002352
        {"type": "ineq", "fun": lambda x: np.sum(returns.mean()*x)-0.003})
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(returns.shape[1]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./returns.shape[1] for x in range(returns.shape[1])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
optimized_results

In [ ]:
weights = optimized_results.x


In [ ]:
np.sum(returns.mean()*optimized_results.x)

In [ ]:
returns.mean()*optimized_results.x

In [ ]:
assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']
assets

In [ ]:
cov_matrix = all_stocks.pct_change().apply(lambda x: np.log(1+x)).cov()

In [ ]:
cov_matrix

In [ ]:
p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

mean = returns.mean()
num_assets = len(symbols)
num_portfolios = 10

for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    sim_returns = np.dot(weights, mean) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(sim_returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

In [ ]:
data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(all_stocks.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]

In [ ]:
portfolios  = pd.DataFrame(data)
portfolios


In [ ]:
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])